# 1 часть -- Импорт библиотек

In [1]:
import pandas as pd
import numpy as np

import scipy.stats as sts
import matplotlib.pyplot as plt
import plotly.express as px
 

# 2 часть -- Начало анализа, первый взгялд на данные

In [6]:
begin_dataset = pd.read_csv('JoJo_Bizarre_Adventure_All_Stands_Stats.csv')

print(begin_dataset.info())
display(begin_dataset)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156 entries, 0 to 155
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Stand   156 non-null    object
 1   PWR     156 non-null    object
 2   SPD     156 non-null    object
 3   RNG     156 non-null    object
 4   STA     156 non-null    object
 5   PRC     156 non-null    object
 6   DEV     156 non-null    object
 7   Story   156 non-null    object
dtypes: object(8)
memory usage: 9.9+ KB
None


,Stand,PWR,SPD,RNG,STA,PRC,DEV,Story
0,Anubis,B,B,E,A,E,C,Part 3: Stardust Crusaders
1,Atum,D,C,D,B,D,D,Part 3: Stardust Crusaders
2,Bastet,E,E,B,A,E,E,Part 3: Stardust Crusaders
3,Cream,B,B,D,C,C,D,Part 3: Stardust Crusaders
4,Death Thirteen,C,C,E,B,D,B,Part 3: Stardust Crusaders
...,...,...,...,...,...,...,...,...
151,Nightbird Flying,E,A,A,A,E,E,Light Novel: Purple Haze Feedback
152,Purple Haze Distortion,A,B,E,E,C,unknown,Light Novel: Purple Haze Feedback
153,Rainy Day Dream Away,E,B,A,A,E,E,Light Novel: Purple Haze Feedback
154,Remote Romance,E,D,infinite,B,C,E,Featured
